# <font color='red'>Face mask detection by camera</font>

In [18]:
from keras.models import load_model
import cv2
import numpy as np

In [19]:
model = load_model('model-020.model')

cap = cv2.VideoCapture(0)

# Import cascade file for facial recognition.
face_clsfr = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

labels_dict = {0: 'NO MASK', 1: 'MASK'}
color_dict = {0: (0, 255, 0), 1: (0, 0, 255)}

rect_size = 4

In [20]:
# Checking if a camera can be open.
if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [21]:
# For saving a record.
# Size of an image.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Define the codec and create VideoWriter object.
# The output is stored in 'output.avi' file.
out = cv2.VideoWriter('output.avi',
                      cv2.VideoWriter_fourcc('M','J','P','G'), 20,
                      (frame_width,frame_height))

In [22]:
while True:
    # Capture frame-by-frame.
    ret, img = cap.read()

    # If image is read correctly ret is True.
    if not ret:
        print('Can not show an image. Stop to work...')
        break

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Getting corners around the face.
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5) # 1.3 = scale factor,
                                                      # 5 = minimum neighbor.

    # Drawing bounding box around face.
    for (x, y, w, h) in faces:

        face_img = gray[y: y + w, x: x + w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized,(1, 100, 100, 1))
        result = model.predict(reshaped)

        label = np.argmax(result,axis=1)[0]

        cv2.rectangle(img,(x, y), (x + w, y + h), color_dict[label], 2)
        cv2.rectangle(img, (x, y - 40), (x + w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display the resulting frame.
    cv2.imshow('LIVE', img)

    # ADDED
    if cv2.waitKey(1) == ord('q'):
        break

# When everything done, release the capture.
cap.release()

# Recording a video.
out.release()

# Closes all the frames.
cv2.destroyAllWindows()

1/1 [==============================] - 0s 63ms/step
